# Risk assessment: Extreme precipitation

- A workflow from the CLIMAAX [Handbook](https://handbook.climaax.eu/) and [MULTI_infrastructure](https://github.com/CLIMAAX/MULTI_infrastructure) GitHub repository.
- See our [how to use risk workflows](https://handbook.climaax.eu/notebooks/workflows_how_to.html) page for information on how to run this notebook.

In this notebook, the calculation of risk associated with the variable under examination will be shown. Pandas dataframes will be presented which, in the example, show nine airports - the operator will need to modify the list of airports by adding/modifying names.
The same concept applies to exposure and vulnerability indicators - indicators will be presented that can be expanded or modified by the user.
The only mandatory thing is that the framework was conceptualized for at least two airports 

In [3]:
import os
import pandas as pd
import numpy as np

## Exposure and Vulnerability

In [4]:
# Define the airports
airports = ['Milano Malpensa', 'Milano Linate', 'Bergamo Orio al Serio', 'Roma Fiumicino', 'Roma Ciampino', 
            'Napoli Capodichino', 'Palermo Punta Raisi', 'Catania Fontanarossa', 'Cagliari Elmas']

In [5]:
csv_folder_path = '/work/cmcc/dg07124/climax/indicators/cordex2/vulnerability_exposure'

if not os.path.exists(csv_folder_path):
    os.makedirs(csv_folder_path)

### Air and Land-Side Components

The various airports components, divided into landside and airside areas, were considered as exposed samples in accordance with De Vivo et al. (2022). More specifically, the air side components include the structures used for the movement of aircraft, such as runways, taxiways, tower, and aprons. On the other hand, the land side components refer to the public access areas such as offices, terminals, airports access systems and parking areas.


| Air side  |Unit of measurement|
|-----------|------------------ |
| Runaway   | Area [m^2]        | 
| Taxiways  | Area [m]          | 
| Tower     | Height [m]        | 
| Apron     | Area [m^2]        | 


--------------------------------------------------------


| Land side                    |Unit of measurement|
|------------------------------|------------------ |
| Terminal                     | Area [m^2]        | 
| Offices and other buildings  | Area [m^2]        | 
| Airport_accesses_systems     | Area [m^2]        | 
| Carparks                     | Number            | 



In [6]:
# Define the values for each indicator and airport
air_land_side_components = {
    'Runways': [470400, 159742, 145530, 396000, 9900, 118260, 292650, 109575, 126000],
    'Taxiways': [250.5, 156, 183, 300, 301, 99, 368, 159, 604],
    'Tower': [80, 47, 30, 56, 60, 40, 25, 12, 28],
    'Apron': [1319000, 387000, 190000, 797250, None, 200000, 148000, 180000, 156000],
    'Terminals': [315000, 85050, 53025, 354300, 20950, 30700, 35400, 43310, 41290],
    'Offices_and_other_buildings': [26715, 13527, 7075, 29520, 5950, 1915, 4180, 6920, 5030],
    'Airport_accesses_systems': [31660, None, None, None, None, None, None, None, None],
    'Carparks': [15000, 3000, 8000, 20100, 1220, 1500, 1364, 1800, 2133]
}

The aim of normalization processes is to transform the values of these indicators, which are measured at different scales and in different units, into comparable values considered on a common scale (Master Adapt, 2018). We used the Min-Max method to transform all values to scores in a range from 0 to 1, where the value 0 represents the optimal level, while the value 1 reflects the most critical estimates:

In [7]:
# Function to normalize data while handling NaN values and ZeroDivisionError
def normalize_data(indicator_values, airports):
    normalized_data = {}

    for indicator, values in indicator_values.items():
        # Filter out None (NaN) values for normalization
        valid_values = [value for value in values if value is not None]

        # Check if we have more than one unique value for normalization
        if len(valid_values) > 1:
            # Get the min and max values for the valid values
            min_val = min(valid_values)
            max_val = max(valid_values)

            # Check for zero division (when min == max)
            if min_val == max_val:
                normalized_values = [None if value is not None else None for value in values]
            else:
                # Normalize the values
                normalized_values = [
                    (value - min_val) / (max_val - min_val) if value is not None else None
                    for value in values
                ]
        else:
            # If all values are the same or all values are NaN, set all normalized values to NaN
            normalized_values = [None if value is not None else None for value in values]

        normalized_data[indicator] = normalized_values

    return normalized_data


In [8]:
# Dictionary to store the dynamic DataFrames
dynamic_dataframes = {}

# Function to display data in a DataFrame format
def display_data(indicator_values, airports, df_name):
    # Create a DataFrame from the original data
    df = pd.DataFrame(indicator_values, index=airports)
    print("Original Data:")
    display(df)

    # Normalize the data
    normalized_data = normalize_data(indicator_values, airports)

    # Convert the normalized data to a DataFrame
    normalized_df = pd.DataFrame(normalized_data, index=airports)
    # Calculate the average normalized value for each airport
    normalized_df['average'] = normalized_df.mean(axis=1)
    print("Normalized Data:")
    display(normalized_df)

    # Separate the "Average" column into another DataFrame
    avg_df = normalized_df[['average']]
    print("Avg components:")
    
    # Store the avg_exposure_df_temp in the dynamic_dataframes dictionary with the provided name
    dynamic_dataframes[df_name] = avg_df
    print(f"Stored DataFrame with name: {df_name}")


In [9]:
# Display the data
display_data(air_land_side_components, airports, 'avg_df_exposure')

Original Data:


,Runways,Taxiways,Tower,Apron,Terminals,Offices_and_other_buildings,Airport_accesses_systems,Carparks
Milano Malpensa,470400,250.5,80,1319000.0,315000,26715,31660.0,15000
Milano Linate,159742,156.0,47,387000.0,85050,13527,NaN,3000
Bergamo Orio al Serio,145530,183.0,30,190000.0,53025,7075,NaN,8000
Roma Fiumicino,396000,300.0,56,797250.0,354300,29520,NaN,20100
Roma Ciampino,9900,301.0,60,NaN,20950,5950,NaN,1220
Napoli Capodichino,118260,99.0,40,200000.0,30700,1915,NaN,1500
Palermo Punta Raisi,292650,368.0,25,148000.0,35400,4180,NaN,1364
Catania Fontanarossa,109575,159.0,12,180000.0,43310,6920,NaN,1800
Cagliari Elmas,126000,604.0,28,156000.0,41290,5030,NaN,2133


Normalized Data:


,Runways,Taxiways,Tower,Apron,Terminals,Offices_and_other_buildings,Airport_accesses_systems,Carparks,average
Milano Malpensa,1.000000,0.300000,1.000000,1.000000,0.882106,0.898388,None,0.729873,0.830052
Milano Linate,0.325390,0.112871,0.514706,0.204099,0.192290,0.420648,None,0.094280,0.266326
Bergamo Orio al Serio,0.294528,0.166337,0.264706,0.035867,0.096220,0.186923,None,0.359110,0.200527
Roma Fiumicino,0.838436,0.398020,0.647059,0.554441,1.000000,1.000000,None,1.000000,0.776851
Roma Ciampino,0.000000,0.400000,0.705882,NaN,0.000000,0.146169,None,0.000000,0.208675
Napoli Capodichino,0.235309,0.000000,0.411765,0.044406,0.029249,0.000000,None,0.014831,0.10508
Palermo Punta Raisi,0.614007,0.532673,0.191176,0.000000,0.043348,0.082050,None,0.007627,0.210126
Catania Fontanarossa,0.216450,0.118812,0.000000,0.027327,0.067077,0.181308,None,0.030720,0.09167
Cagliari Elmas,0.252117,1.000000,0.235294,0.006832,0.061017,0.112842,None,0.048358,0.245209


Avg components:
Stored DataFrame with name: avg_df_exposure


In [10]:
# Accessing the avg_exposure_temp DataFrames
avg_df_exposure = dynamic_dataframes['avg_df_exposure']
display(avg_df_exposure)

,average
Milano Malpensa,0.830052
Milano Linate,0.266326
Bergamo Orio al Serio,0.200527
Roma Fiumicino,0.776851
Roma Ciampino,0.208675
Napoli Capodichino,0.10508
Palermo Punta Raisi,0.210126
Catania Fontanarossa,0.09167
Cagliari Elmas,0.245209


## Calculation of Vulnerability

Vulnerability reflects “the propensity or predisposition of a system to be adversely affected. It encompasses a variety of concepts and elements including sensitivity or susceptibility to harm and lack of capacity to cope and adapt opportunities, or to respond to consequences” (IPCC, 2022).

Vulnerability is composed by sensitivity and adaptive capacity


### Sensitivity

Sensitivity is intended as the degree to which the system is (positively or negatively) affected by climate variability or climate change. 


| Indicator                     |Unit of measurement                |
|-------------------------------|-----------------------------------|
| Soil Sealing                  | Area [%]                          | 
| Passenger                     | Number                            | 
| Building in bad condition     | Number or Absence/Presence        | 
| Age building                  | Age                               | 
| Air traffic                   | Number                            |
| Parking accesses              | Number                            |
| Staff work outside airport    | Number                            |
| Underground infrastructures   | Number (or Aerea)                 |
| Flooded areas                 | Number (or %)                     |

In [20]:
# Define the sensitivity indicators and their values
sensitivity_indicators = {
    'Soil_sealing': [1235, 300, 300, 1590, 133, 217, 391, 225, 246],
    'Passengers': [268000000, 65000000 , 13857257, 43532573, 5879496, 10860068, 7018087, 10223113, 4747806],
    'Buildings_bad_conditions_or_maintenance': [None, None, None, None, None, None, None, None, None],
    'Age_buildings': [73, 82, 83, 63, 105, 111, 61, 97, 84],
    'Air_traffic': [201050, 85730, 95377, 309783, 52253, 82577, 54243, 73494, 39691],
    'Underground_infrastructures': [30300, 14700, 5700, 26250, 2250, 0, 0, 8650, 0],
    'Flooded_areas': [None, None, None, None, None, None, None, None, None]
}

In [21]:
# Execute the function to display the data for Sensitivity Indicators
display_data(sensitivity_indicators, airports, 'avg_df_sensitivity')

Original Data:


,Soil_sealing,Passengers,Buildings_bad_conditions_or_maintenance,Age_buildings,Air_traffic,Underground_infrastructures,Flooded_areas
Milano Malpensa,1235,268000000,None,73,201050,30300,None
Milano Linate,300,65000000,None,82,85730,14700,None
Bergamo Orio al Serio,300,13857257,None,83,95377,5700,None
Roma Fiumicino,1590,43532573,None,63,309783,26250,None
Roma Ciampino,133,5879496,None,105,52253,2250,None
Napoli Capodichino,217,10860068,None,111,82577,0,None
Palermo Punta Raisi,391,7018087,None,61,54243,0,None
Catania Fontanarossa,225,10223113,None,97,73494,8650,None
Cagliari Elmas,246,4747806,None,84,39691,0,None


Normalized Data:


,Soil_sealing,Passengers,Buildings_bad_conditions_or_maintenance,Age_buildings,Air_traffic,Underground_infrastructures,Flooded_areas,average
Milano Malpensa,0.756349,1.000000,None,0.24,0.597422,1.000000,None,0.718754
Milano Linate,0.114619,0.228876,None,0.42,0.170457,0.485149,None,0.28382
Bergamo Orio al Serio,0.114619,0.034604,None,0.44,0.206174,0.188119,None,0.196703
Roma Fiumicino,1.000000,0.147329,None,0.04,1.000000,0.866337,None,0.610733
Roma Ciampino,0.000000,0.004299,None,0.88,0.046510,0.074257,None,0.201013
Napoli Capodichino,0.057653,0.023218,None,1.00,0.158783,0.000000,None,0.247931
Palermo Punta Raisi,0.177076,0.008624,None,0.00,0.053878,0.000000,None,0.047916
Catania Fontanarossa,0.063143,0.020799,None,0.72,0.125154,0.285479,None,0.242915
Cagliari Elmas,0.077557,0.000000,None,0.46,0.000000,0.000000,None,0.107511


Avg components:
Stored DataFrame with name: avg_df_sensitivity


In [23]:
# Accessing the avg_exposure_temp DataFrames
avg_df_sensitivity = dynamic_dataframes['avg_df_sensitivity']
display(avg_df_sensitivity)

,average
Milano Malpensa,0.718754
Milano Linate,0.28382
Bergamo Orio al Serio,0.196703
Roma Fiumicino,0.610733
Roma Ciampino,0.201013
Napoli Capodichino,0.247931
Palermo Punta Raisi,0.047916
Catania Fontanarossa,0.242915
Cagliari Elmas,0.107511


## Adaptative capacity

Adaptive capacity is “the ability of systems, institutions, humans and other organisms to adjust to potential damage, to take advantage of

ollowing the list of Adaptive capacity indicator considered in this work
This indicatori are defined by the classes reported below and included in De Vivo et al. (2021), if the indicator is presence/absence its value will be 0.9/0.1 respectively 


- Risk awareness: Initiatives for mitigation to climate change:
 1. Adherence to "NetZero2050" Programme (neutrality3+)
 2. Certification ISO 50001 and initiatives of "energy saving"
 3. Neutrality 3+ Airport carbon accreditation
 4. Neutrality 4+ Airport carbon accreditation; EP-100 intelligent use of energy from "The Climate Group"- Certification ISO 50001
 5. Level 2 Airport carbon accreditation, environmental improvement program
 6. Commitment to obtaining  certfiication 

Initiatives for adaptation to climate change:
- Green wall
 1. Vertical outdoor green wall
 2. Absent
- Green roof
 1. Roof Garden
 2. Absent
- Heat resistent coating
 1. Absent
 2. Graphene asphalt
  
- Insurance policy for extreme events
 1. Absent
- Guidelines for adaptation plan to climate change
 1. plan (PACC) and regional strategy for  adaptation to climate change
 2. Regional plan under development
 3. Absent
 4. Sustainable Energy and Climate Action Plan (Covenant of Mayors)
 5. Sustainable Energy and Climate Action Plan (Covenant of Mayors) municipality of Catania
 6. Regional climate change adaptation strategy
- Evening departures
 1. Absent

| Class No. | Description      | Value Rane | Indicator Range (0-1) |
|-----------|------------------|-------------|-----------------------|
| 1         | Optimal          | 0 - 0.2     | 0.1                   |
| 2         | Quite Positive   | 0.2 - 0.4   | 0.3                   |
| 3         | Neutral          | 0.4 - 0.6   | 0.5                   |
| 4         | Quite Negative   | 0.6 - 0.8   | 0.7                   |
| 5         | Critical         | 0.8 - 1     | 0.9                   |


In [24]:
# Define the sensitivity indicators and their values
adaptative_indicators = {
    'Vegetation_zone': [None, None, None, None, None, None, None, None, None],
    'Initiatives_mitigation_climate_change_risk_awareness': [0.3, 0.3, 0.3, 0.1, 0.1, 0.3, 0.5, 0.7, 0.7],
    'Efficient_drainage_system': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    'Insurance_policy_for_extreme_events': [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Monitoring_and_alarm_system': [0.1, 0.1, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    'Bioinfiltration_and_permeable_pavements': [0.9, 0.9, 0.1, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Guidelines_for_adaptation_climate_change': [0.3, 0.3, 0.3, 0.5, 0.5, 0.9, 0.3, 0.3, 0.3]
}

In [25]:
# Create the DataFrame for the adaptive indicators
adaptative_df = pd.DataFrame(adaptative_indicators, index=airports)

# Add a column with the average of all indicators for each airport
adaptative_df['average'] = adaptative_df.mean(axis=1)

display(adaptative_df)

# Extract a column the average column
avg_df_adaptative = adaptative_df[['average']]

display(avg_df_adaptative)

,Vegetation_zone,Initiatives_mitigation_climate_change_risk_awareness,Efficient_drainage_system,Insurance_policy_for_extreme_events,Monitoring_and_alarm_system,Bioinfiltration_and_permeable_pavements,Guidelines_for_adaptation_climate_change,average
Milano Malpensa,None,0.3,0.1,0.9,0.1,0.9,0.3,0.433333
Milano Linate,None,0.3,0.1,0.9,0.1,0.9,0.3,0.433333
Bergamo Orio al Serio,None,0.3,0.1,0.9,0.3,0.1,0.3,0.333333
Roma Fiumicino,None,0.1,0.1,0.9,0.3,0.9,0.5,0.466667
Roma Ciampino,None,0.1,0.1,0.9,0.3,0.9,0.5,0.466667
Napoli Capodichino,None,0.3,0.1,0.9,0.3,0.9,0.9,0.566667
Palermo Punta Raisi,None,0.5,0.1,0.9,0.3,0.9,0.3,0.5
Catania Fontanarossa,None,0.7,0.1,0.9,0.3,0.9,0.3,0.533333
Cagliari Elmas,None,0.7,0.1,0.9,0.3,0.9,0.3,0.533333


,average
Milano Malpensa,0.433333
Milano Linate,0.433333
Bergamo Orio al Serio,0.333333
Roma Fiumicino,0.466667
Roma Ciampino,0.466667
Napoli Capodichino,0.566667
Palermo Punta Raisi,0.5
Catania Fontanarossa,0.533333
Cagliari Elmas,0.533333


## Vulnerability calculation
As mentioned before, the Vulnerability is the mean between the sensitivity and adaptive capacity

In [26]:
vulnerability_df = pd.DataFrame()
# Combine the 'Average' columns from both DataFrames and calculate the new average
vulnerability_df['average'] = (avg_df_sensitivity['average'] + avg_df_adaptative['average']) / 2

normalized_avg_path = os.path.join(csv_folder_path, f'avg_df_vulnerability_prep.csv')
vulnerability_df.to_csv(normalized_avg_path)

# Display the resulting vulnerability DataFrame
vulnerability_df

,average
Milano Malpensa,0.576044
Milano Linate,0.358577
Bergamo Orio al Serio,0.265018
Roma Fiumicino,0.5387
Roma Ciampino,0.33384
Napoli Capodichino,0.407299
Palermo Punta Raisi,0.273958
Catania Fontanarossa,0.388124
Cagliari Elmas,0.320422


### Risk Calculation for Extreme Precipitation

The risk will be computed by moltiplication of three components: 
R = H * E * V

### UERRA DATASETS:

In the cell below the nearest point to the airport was selected. In the framework the normalization of all the indicator was made, then a mean to obtain a single value


- **Extreme Precipitation**

In [27]:
hazard_uerra_prep_path = '/work/cmcc/dg07124/climax/indicators/hazards_csv/hazard_uerra_precip.csv'
hazard_uerra_prep = pd.read_csv(hazard_uerra_prep_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_uerra_prep.index = hazard_uerra_prep.index.str.title()
hazard_uerra_prep['average'] = pd.to_numeric(hazard_uerra_prep['average'], errors='coerce')
hazard_uerra_prep = hazard_uerra_prep.sort_index()
hazard_uerra_prep

,average
Airports,
Bergamo Orio Al Serio,13.090606
Cagliari Elmas,15.807716
Catania Fontanarossa,17.416528
Milano Linate,13.321168
Milano Malpensa,12.487971
Napoli Capodichino,14.670617
Palermo Punta Raisi,13.622919
Roma Ciampino,14.587898
Roma Fiumicino,12.443477


In [29]:
vulnerability_df.index = vulnerability_df.index.str.title()
vulnerability_df['average'] = pd.to_numeric(vulnerability_df['average'], errors='coerce')
vulnerability_df = vulnerability_df.sort_index()
vulnerability_df

,average
Bergamo Orio Al Serio,0.265018
Cagliari Elmas,0.320422
Catania Fontanarossa,0.388124
Milano Linate,0.358577
Milano Malpensa,0.576044
Napoli Capodichino,0.407299
Palermo Punta Raisi,0.273958
Roma Ciampino,0.333840
Roma Fiumicino,0.538700


In [30]:
avg_df_exposure.index = avg_df_exposure.index.str.title()
avg_df_exposure['average'] = pd.to_numeric(avg_df_exposure['average'], errors='coerce')
avg_df_exposure = avg_df_exposure.sort_index()
avg_df_exposure

,average
Bergamo Orio Al Serio,0.200527
Cagliari Elmas,0.245209
Catania Fontanarossa,0.091670
Milano Linate,0.266326
Milano Malpensa,0.830052
Napoli Capodichino,0.105080
Palermo Punta Raisi,0.210126
Roma Ciampino,0.208675
Roma Fiumicino,0.776851


In [31]:
# Now multiply the 'average' column of each DataFrame
uerra_risk_prep = avg_df_exposure['average'] * hazard_uerra_prep['average'] * vulnerability_df['average']
# Convert the Series to a DataFrame
uerra_risk_prep_df = pd.DataFrame(uerra_risk_prep)

# Rename the column
uerra_risk_prep_df.columns = ['Uerra Risk Precipitation']
uerra_risk_prep_df

,Uerra Risk Precipitation
Bergamo Orio Al Serio,0.695679
Cagliari Elmas,1.242017
Catania Fontanarossa,0.619672
Milano Linate,1.272151
Milano Malpensa,5.971080
Napoli Capodichino,0.627887
Palermo Punta Raisi,0.784212
Roma Ciampino,1.016253
Roma Fiumicino,5.207464


## RISK CLASSES

To evaluate the risk classes for each of the airport the Quantile Method was used. Classifying each airport in terms of quantile with respect to all other airport taken into account

In [32]:
# Function to calculate the quantile of the risk for all 9 airports
def classify_dataframe(df, column_name):
    """
    Classifies a given dataframe column into quantiles and assigns categorical classes.
    
    Parameters:
    df (pd.DataFrame): Input dataframe with numerical values.
    column_name (str): The column name to be classified.
    
    Returns:
    pd.DataFrame: Dataframe with quantile values and class categories.
    """
    df = df.copy()  # Ensure the original dataframe is not modified
    
    # Calculating the quantiles on a scale from 0 to 1
    df['Quantile'] = df[column_name].rank(pct=True)
    
    # Defining classification labels
    labels = ["Very Low", "Low", "Medium", "High", "Very High"]
    
    # Assigning classes based on quantile values
    df['Class'] = pd.qcut(df['Quantile'], q=5, labels=labels)
    return df

In [33]:
# Define a function to color the class column based on the classes
def highlight_class_column(s):
    colors = {
        "Very Low": "background-color: forestgreen",
        "Low": "background-color: limegreen",
        "Medium": "background-color: yellow",
        "High": "background-color: orange",
        "Very High": "background-color: red; color: white"
    }
    return [colors.get(val, "") if s.name == "Class" else "" for val in s]

In [34]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(uerra_risk_prep_df, "Uerra Risk Precipitation")

# Apply styling
styled_uerra_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_uerra_risk_prep_df)

,Uerra Risk Precipitation,Quantile,Class
Bergamo Orio Al Serio,0.695679,0.333333,Low
Cagliari Elmas,1.242017,0.666667,High
Catania Fontanarossa,0.619672,0.111111,Very Low
Milano Linate,1.272151,0.777778,High
Milano Malpensa,5.971080,1.000000,Very High
Napoli Capodichino,0.627887,0.222222,Very Low
Palermo Punta Raisi,0.784212,0.444444,Low
Roma Ciampino,1.016253,0.555556,Medium
Roma Fiumicino,5.207464,0.888889,Very High


The Risk computation for Cordex dataset follows the same steps as UERRA

## CORDEX Dataset

### RCP26 2021-2050

In [35]:
hazard_cordex_prep_rcp26_20212050_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp26_2021-2050_data.csv'
hazard_cordex_prep_rcp26_20212050 = pd.read_csv(hazard_cordex_prep_rcp26_20212050_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp26_20212050.index = hazard_cordex_prep_rcp26_20212050.index.str.title()
hazard_cordex_prep_rcp26_20212050['average'] = pd.to_numeric(hazard_cordex_prep_rcp26_20212050['average'], errors='coerce')
hazard_cordex_prep_rcp26_20212050 = hazard_cordex_prep_rcp26_20212050.sort_index()
hazard_cordex_prep_rcp26_20212050 = hazard_cordex_prep_rcp26_20212050[['average']]
hazard_cordex_prep_rcp26_20212050

,average
Airports,
Bergamo Orio Al Serio,0.839782
Cagliari Elmas,0.129712
Catania Fontanarossa,0.202637
Milano Linate,0.680490
Milano Malpensa,0.498172
Napoli Capodichino,0.531602
Palermo Punta Raisi,0.400203
Roma Ciampino,0.762894
Roma Fiumicino,0.971122


In [36]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp26_20212050 = avg_df_exposure['average'] * hazard_cordex_prep_rcp26_20212050['average'] * vulnerability_df['average']
cordex_risk_prep_rcp26_20212050

# Convert the Series to a DataFrame
cordex_risk_prep_rcp26_20212050_df = pd.DataFrame(cordex_risk_prep_rcp26_20212050)

# Rename the column
cordex_risk_prep_rcp26_20212050_df.columns = ['Cordex Risk Prep rcp26 2021-2050']
cordex_risk_prep_rcp26_20212050_df

,Cordex Risk Prep rcp26 2021-2050
Bergamo Orio Al Serio,0.044629
Cagliari Elmas,0.010191
Catania Fontanarossa,0.007210
Milano Linate,0.064986
Milano Malpensa,0.238199
Napoli Capodichino,0.022752
Palermo Punta Raisi,0.023038
Roma Ciampino,0.053146
Roma Fiumicino,0.406404


In [37]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp26_20212050_df, "Cordex Risk Prep rcp26 2021-2050")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp26 2021-2050,Quantile,Class
Bergamo Orio Al Serio,0.044629,0.555556,Medium
Cagliari Elmas,0.010191,0.222222,Very Low
Catania Fontanarossa,0.007210,0.111111,Very Low
Milano Linate,0.064986,0.777778,High
Milano Malpensa,0.238199,0.888889,Very High
Napoli Capodichino,0.022752,0.333333,Low
Palermo Punta Raisi,0.023038,0.444444,Low
Roma Ciampino,0.053146,0.666667,High
Roma Fiumicino,0.406404,1.000000,Very High


### RCP26 2041-2070

In [38]:
hazard_cordex_prep_rcp26_20412070_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp26_2041-2070_data.csv'
hazard_cordex_prep_rcp26_20412070 = pd.read_csv(hazard_cordex_prep_rcp26_20412070_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp26_20412070.index = hazard_cordex_prep_rcp26_20412070.index.str.title()
hazard_cordex_prep_rcp26_20412070['average'] = pd.to_numeric(hazard_cordex_prep_rcp26_20412070['average'], errors='coerce')
hazard_cordex_prep_rcp26_20412070 = hazard_cordex_prep_rcp26_20412070.sort_index()
hazard_cordex_prep_rcp26_20412070 = hazard_cordex_prep_rcp26_20412070[['average']]
hazard_cordex_prep_rcp26_20412070

,average
Airports,
Bergamo Orio Al Serio,0.887246
Cagliari Elmas,0.546337
Catania Fontanarossa,0.000000
Milano Linate,0.728009
Milano Malpensa,0.706074
Napoli Capodichino,0.667614
Palermo Punta Raisi,0.548515
Roma Ciampino,0.751005
Roma Fiumicino,0.916865


In [39]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp26_20412070 = avg_df_exposure['average'] * hazard_cordex_prep_rcp26_20412070['average'] * vulnerability_df['average']
cordex_risk_prep_rcp26_20412070

# Convert the Series to a DataFrame
cordex_risk_prep_rcp26_20412070_df = pd.DataFrame(cordex_risk_prep_rcp26_20412070)

# Rename the column
cordex_risk_prep_rcp26_20412070_df.columns = ['Cordex Risk Prep rcp26 2041-2070']
cordex_risk_prep_rcp26_20412070_df

,Cordex Risk Prep rcp26 2041-2070
Bergamo Orio Al Serio,0.047151
Cagliari Elmas,0.042926
Catania Fontanarossa,0.000000
Milano Linate,0.069524
Milano Malpensa,0.337607
Napoli Capodichino,0.028573
Palermo Punta Raisi,0.031576
Roma Ciampino,0.052318
Roma Fiumicino,0.383698


In [40]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp26_20412070_df, "Cordex Risk Prep rcp26 2041-2070")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp26 2041-2070,Quantile,Class
Bergamo Orio Al Serio,0.047151,0.555556,Medium
Cagliari Elmas,0.042926,0.444444,Low
Catania Fontanarossa,0.000000,0.111111,Very Low
Milano Linate,0.069524,0.777778,High
Milano Malpensa,0.337607,0.888889,Very High
Napoli Capodichino,0.028573,0.222222,Very Low
Palermo Punta Raisi,0.031576,0.333333,Low
Roma Ciampino,0.052318,0.666667,High
Roma Fiumicino,0.383698,1.000000,Very High


### RCP26 2071-2100

In [41]:
hazard_cordex_prep_rcp26_20712100_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp26_2071-2100_data.csv'
hazard_cordex_prep_rcp26_20712100 = pd.read_csv(hazard_cordex_prep_rcp26_20712100_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp26_20712100.index = hazard_cordex_prep_rcp26_20712100.index.str.title()
hazard_cordex_prep_rcp26_20712100['average'] = pd.to_numeric(hazard_cordex_prep_rcp26_20712100['average'], errors='coerce')
hazard_cordex_prep_rcp26_20712100 = hazard_cordex_prep_rcp26_20712100.sort_index()
hazard_cordex_prep_rcp26_20712100 = hazard_cordex_prep_rcp26_20712100[['average']]
hazard_cordex_prep_rcp26_20712100

,average
Airports,
Bergamo Orio Al Serio,0.721032
Cagliari Elmas,0.152600
Catania Fontanarossa,0.238316
Milano Linate,0.580239
Milano Malpensa,0.266017
Napoli Capodichino,0.327587
Palermo Punta Raisi,0.771008
Roma Ciampino,0.833393
Roma Fiumicino,0.828560


In [42]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp26_20712100 = avg_df_exposure['average'] * hazard_cordex_prep_rcp26_20712100['average'] * vulnerability_df['average']
cordex_risk_prep_rcp26_20712100

# Convert the Series to a DataFrame
cordex_risk_prep_rcp26_20712100_df = pd.DataFrame(cordex_risk_prep_rcp26_20712100)

# Rename the column
cordex_risk_prep_rcp26_20712100_df.columns = ['Cordex Risk Prep rcp26 2071-2100']
cordex_risk_prep_rcp26_20712100_df

,Cordex Risk Prep rcp26 2071-2100
Bergamo Orio Al Serio,0.038318
Cagliari Elmas,0.011990
Catania Fontanarossa,0.008479
Milano Linate,0.055412
Milano Malpensa,0.127195
Napoli Capodichino,0.014020
Palermo Punta Raisi,0.044384
Roma Ciampino,0.058058
Roma Fiumicino,0.346744


In [43]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp26_20712100_df, "Cordex Risk Prep rcp26 2071-2100")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp26 2071-2100,Quantile,Class
Bergamo Orio Al Serio,0.038318,0.444444,Low
Cagliari Elmas,0.011990,0.222222,Very Low
Catania Fontanarossa,0.008479,0.111111,Very Low
Milano Linate,0.055412,0.666667,High
Milano Malpensa,0.127195,0.888889,Very High
Napoli Capodichino,0.014020,0.333333,Low
Palermo Punta Raisi,0.044384,0.555556,Medium
Roma Ciampino,0.058058,0.777778,High
Roma Fiumicino,0.346744,1.000000,Very High


### RCP45 2021-2050

In [44]:
hazard_cordex_prep_rcp45_20212050_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp45_2021-2050_data.csv'
hazard_cordex_prep_rcp45_20212050 = pd.read_csv(hazard_cordex_prep_rcp45_20212050_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp45_20212050.index = hazard_cordex_prep_rcp45_20212050.index.str.title()
hazard_cordex_prep_rcp45_20212050['average'] = pd.to_numeric(hazard_cordex_prep_rcp45_20212050['average'], errors='coerce')
hazard_cordex_prep_rcp45_20212050 = hazard_cordex_prep_rcp45_20212050.sort_index()
hazard_cordex_prep_rcp45_20212050 = hazard_cordex_prep_rcp45_20212050[['average']]
hazard_cordex_prep_rcp45_20212050

,average
Airports,
Bergamo Orio Al Serio,0.791883
Cagliari Elmas,0.212627
Catania Fontanarossa,0.091229
Milano Linate,0.360072
Milano Malpensa,0.576764
Napoli Capodichino,0.264596
Palermo Punta Raisi,0.107250
Roma Ciampino,0.956627
Roma Fiumicino,0.795725


In [45]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp45_20212050 = avg_df_exposure['average'] * hazard_cordex_prep_rcp45_20212050['average'] * vulnerability_df['average']
cordex_risk_prep_rcp45_20212050

# Convert the Series to a DataFrame
cordex_risk_prep_rcp45_20212050_df = pd.DataFrame(cordex_risk_prep_rcp45_20212050)

# Rename the column
cordex_risk_prep_rcp45_20212050_df.columns = ['Cordex Risk Prep rcp45 2021-2050']
cordex_risk_prep_rcp45_20212050_df

,Cordex Risk Prep rcp45 2021-2050
Bergamo Orio Al Serio,0.042083
Cagliari Elmas,0.016706
Catania Fontanarossa,0.003246
Milano Linate,0.034386
Milano Malpensa,0.275778
Napoli Capodichino,0.011324
Palermo Punta Raisi,0.006174
Roma Ciampino,0.066643
Roma Fiumicino,0.333003


In [46]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp45_20212050_df, "Cordex Risk Prep rcp45 2021-2050")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp45 2021-2050,Quantile,Class
Bergamo Orio Al Serio,0.042083,0.666667,High
Cagliari Elmas,0.016706,0.444444,Low
Catania Fontanarossa,0.003246,0.111111,Very Low
Milano Linate,0.034386,0.555556,Medium
Milano Malpensa,0.275778,0.888889,Very High
Napoli Capodichino,0.011324,0.333333,Low
Palermo Punta Raisi,0.006174,0.222222,Very Low
Roma Ciampino,0.066643,0.777778,High
Roma Fiumicino,0.333003,1.000000,Very High


### RCP45 2041-2070

In [47]:
hazard_cordex_prep_rcp45_20412070_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp45_2041-2070_data.csv'
hazard_cordex_prep_rcp45_20412070 = pd.read_csv(hazard_cordex_prep_rcp45_20412070_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp45_20412070.index = hazard_cordex_prep_rcp45_20412070.index.str.title()
hazard_cordex_prep_rcp45_20412070['average'] = pd.to_numeric(hazard_cordex_prep_rcp45_20412070['average'], errors='coerce')
hazard_cordex_prep_rcp45_20412070 = hazard_cordex_prep_rcp45_20412070.sort_index()
hazard_cordex_prep_rcp45_20412070 = hazard_cordex_prep_rcp45_20412070[['average']]
hazard_cordex_prep_rcp45_20412070

,average
Airports,
Bergamo Orio Al Serio,0.639665
Cagliari Elmas,0.107101
Catania Fontanarossa,0.686725
Milano Linate,0.404216
Milano Malpensa,0.162936
Napoli Capodichino,0.443863
Palermo Punta Raisi,0.652522
Roma Ciampino,0.935870
Roma Fiumicino,0.745211


In [48]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp45_20412070 = avg_df_exposure['average'] * hazard_cordex_prep_rcp45_20412070['average'] * vulnerability_df['average']
cordex_risk_prep_rcp45_20412070

# Convert the Series to a DataFrame
cordex_risk_prep_rcp45_20412070_df = pd.DataFrame(cordex_risk_prep_rcp45_20412070)

# Rename the column
cordex_risk_prep_rcp45_20412070_df.columns = ['Cordex Risk Prep rcp45 2041-2070']
cordex_risk_prep_rcp45_20412070_df

,Cordex Risk Prep rcp45 2041-2070
Bergamo Orio Al Serio,0.033994
Cagliari Elmas,0.008415
Catania Fontanarossa,0.024433
Milano Linate,0.038602
Milano Malpensa,0.077907
Napoli Capodichino,0.018997
Palermo Punta Raisi,0.037563
Roma Ciampino,0.065197
Roma Fiumicino,0.311863


In [49]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp45_20412070_df, "Cordex Risk Prep rcp45 2041-2070")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp45 2041-2070,Quantile,Class
Bergamo Orio Al Serio,0.033994,0.444444,Low
Cagliari Elmas,0.008415,0.111111,Very Low
Catania Fontanarossa,0.024433,0.333333,Low
Milano Linate,0.038602,0.666667,High
Milano Malpensa,0.077907,0.888889,Very High
Napoli Capodichino,0.018997,0.222222,Very Low
Palermo Punta Raisi,0.037563,0.555556,Medium
Roma Ciampino,0.065197,0.777778,High
Roma Fiumicino,0.311863,1.000000,Very High


### RCP45 2071-2100

In [50]:
hazard_cordex_prep_rcp45_20712100_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp45_2071-2100_data.csv'
hazard_cordex_prep_rcp45_20712100 = pd.read_csv(hazard_cordex_prep_rcp45_20712100_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp45_20712100.index = hazard_cordex_prep_rcp45_20712100.index.str.title()
hazard_cordex_prep_rcp45_20712100['average'] = pd.to_numeric(hazard_cordex_prep_rcp45_20712100['average'], errors='coerce')
hazard_cordex_prep_rcp45_20712100 = hazard_cordex_prep_rcp45_20712100.sort_index()
hazard_cordex_prep_rcp45_20712100 = hazard_cordex_prep_rcp45_20712100[['average']]
hazard_cordex_prep_rcp45_20712100

,average
Airports,
Bergamo Orio Al Serio,0.694989
Cagliari Elmas,0.207298
Catania Fontanarossa,0.731509
Milano Linate,0.479583
Milano Malpensa,0.518685
Napoli Capodichino,0.213635
Palermo Punta Raisi,0.427578
Roma Ciampino,0.901619
Roma Fiumicino,0.882974


In [51]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp45_20712100 = avg_df_exposure['average'] * hazard_cordex_prep_rcp45_20712100['average'] * vulnerability_df['average']
cordex_risk_prep_rcp45_20712100

# Convert the Series to a DataFrame
cordex_risk_prep_rcp45_20712100_df = pd.DataFrame(cordex_risk_prep_rcp45_20712100)

# Rename the column
cordex_risk_prep_rcp45_20712100_df.columns = ['Cordex Risk Prep rcp45 2071-2100']
cordex_risk_prep_rcp45_20712100_df

,Cordex Risk Prep rcp45 2071-2100
Bergamo Orio Al Serio,0.036934
Cagliari Elmas,0.016287
Catania Fontanarossa,0.026027
Milano Linate,0.045799
Milano Malpensa,0.248008
Napoli Capodichino,0.009143
Palermo Punta Raisi,0.024614
Roma Ciampino,0.062811
Roma Fiumicino,0.369515


In [52]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp45_20712100_df, "Cordex Risk Prep rcp45 2071-2100")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp45 2071-2100,Quantile,Class
Bergamo Orio Al Serio,0.036934,0.555556,Medium
Cagliari Elmas,0.016287,0.222222,Very Low
Catania Fontanarossa,0.026027,0.444444,Low
Milano Linate,0.045799,0.666667,High
Milano Malpensa,0.248008,0.888889,Very High
Napoli Capodichino,0.009143,0.111111,Very Low
Palermo Punta Raisi,0.024614,0.333333,Low
Roma Ciampino,0.062811,0.777778,High
Roma Fiumicino,0.369515,1.000000,Very High


### RCP85 2021-2050

In [53]:
hazard_cordex_prep_rcp85_20212050_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp85_2021-2050_data.csv'
hazard_cordex_prep_rcp85_20212050 = pd.read_csv(hazard_cordex_prep_rcp85_20212050_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp85_20212050.index = hazard_cordex_prep_rcp85_20212050.index.str.title()
hazard_cordex_prep_rcp85_20212050['average'] = pd.to_numeric(hazard_cordex_prep_rcp85_20212050['average'], errors='coerce')
hazard_cordex_prep_rcp85_20212050 = hazard_cordex_prep_rcp85_20212050.sort_index()
hazard_cordex_prep_rcp85_20212050 = hazard_cordex_prep_rcp85_20212050[['average']]
hazard_cordex_prep_rcp85_20212050

,average
Airports,
Bergamo Orio Al Serio,0.527761
Cagliari Elmas,0.037320
Catania Fontanarossa,0.404908
Milano Linate,0.471399
Milano Malpensa,0.296800
Napoli Capodichino,0.471499
Palermo Punta Raisi,0.502301
Roma Ciampino,0.882944
Roma Fiumicino,0.978143


In [54]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp85_20212050 = avg_df_exposure['average'] * hazard_cordex_prep_rcp85_20212050['average'] * vulnerability_df['average']
cordex_risk_prep_rcp85_20212050

# Convert the Series to a DataFrame
cordex_risk_prep_rcp85_20212050_df = pd.DataFrame(cordex_risk_prep_rcp85_20212050)

# Rename the column
cordex_risk_prep_rcp85_20212050_df.columns = ['Cordex Risk Prep rcp85 2021-2050']
cordex_risk_prep_rcp85_20212050_df

,Cordex Risk Prep rcp85 2021-2050
Bergamo Orio Al Serio,0.028047
Cagliari Elmas,0.002932
Catania Fontanarossa,0.014406
Milano Linate,0.045018
Milano Malpensa,0.141914
Napoli Capodichino,0.020180
Palermo Punta Raisi,0.028915
Roma Ciampino,0.061510
Roma Fiumicino,0.409343


In [55]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp85_20212050_df, "Cordex Risk Prep rcp85 2021-2050")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp85 2021-2050,Quantile,Class
Bergamo Orio Al Serio,0.028047,0.444444,Low
Cagliari Elmas,0.002932,0.111111,Very Low
Catania Fontanarossa,0.014406,0.222222,Very Low
Milano Linate,0.045018,0.666667,High
Milano Malpensa,0.141914,0.888889,Very High
Napoli Capodichino,0.020180,0.333333,Low
Palermo Punta Raisi,0.028915,0.555556,Medium
Roma Ciampino,0.061510,0.777778,High
Roma Fiumicino,0.409343,1.000000,Very High


### RCP85 2041-2070

In [56]:
hazard_cordex_prep_rcp85_20412070_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp85_2041-2070_data.csv'
hazard_cordex_prep_rcp85_20412070 = pd.read_csv(hazard_cordex_prep_rcp85_20412070_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp85_20412070.index = hazard_cordex_prep_rcp85_20412070.index.str.title()
hazard_cordex_prep_rcp85_20412070['average'] = pd.to_numeric(hazard_cordex_prep_rcp85_20412070['average'], errors='coerce')
hazard_cordex_prep_rcp85_20412070 = hazard_cordex_prep_rcp85_20412070.sort_index()
hazard_cordex_prep_rcp85_20412070 = hazard_cordex_prep_rcp85_20412070[['average']]
hazard_cordex_prep_rcp85_20412070

,average
Airports,
Bergamo Orio Al Serio,0.474998
Cagliari Elmas,0.443952
Catania Fontanarossa,0.133516
Milano Linate,0.682315
Milano Malpensa,0.105169
Napoli Capodichino,0.516712
Palermo Punta Raisi,0.434595
Roma Ciampino,0.677381
Roma Fiumicino,0.986059


In [57]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp85_20412070 = avg_df_exposure['average'] * hazard_cordex_prep_rcp85_20412070['average'] * vulnerability_df['average']
cordex_risk_prep_rcp85_20412070

# Convert the Series to a DataFrame
cordex_risk_prep_rcp85_20412070_df = pd.DataFrame(cordex_risk_prep_rcp85_20412070)

# Rename the column
cordex_risk_prep_rcp85_20412070_df.columns = ['Cordex Risk Prep rcp85 2041-2070']
cordex_risk_prep_rcp85_20412070_df

,Cordex Risk Prep rcp85 2041-2070
Bergamo Orio Al Serio,0.025243
Cagliari Elmas,0.034881
Catania Fontanarossa,0.004750
Milano Linate,0.065160
Milano Malpensa,0.050286
Napoli Capodichino,0.022115
Palermo Punta Raisi,0.025018
Roma Ciampino,0.047189
Roma Fiumicino,0.412655


In [58]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp85_20412070_df, "Cordex Risk Prep rcp85 2041-2070")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp85 2041-2070,Quantile,Class
Bergamo Orio Al Serio,0.025243,0.444444,Low
Cagliari Elmas,0.034881,0.555556,Medium
Catania Fontanarossa,0.004750,0.111111,Very Low
Milano Linate,0.065160,0.888889,Very High
Milano Malpensa,0.050286,0.777778,High
Napoli Capodichino,0.022115,0.222222,Very Low
Palermo Punta Raisi,0.025018,0.333333,Low
Roma Ciampino,0.047189,0.666667,High
Roma Fiumicino,0.412655,1.000000,Very High


### RCP85 2071-2100

In [60]:
hazard_cordex_prep_rcp85_20712100_path = '/work/cmcc/dg07124/climax/indicators/cordex2/prep_avg_ensembles/prep_normalized_prep_rcp85_2071-2100_data.csv'
hazard_cordex_prep_rcp85_20712100 = pd.read_csv(hazard_cordex_prep_rcp85_20712100_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_prep_rcp85_20712100.index = hazard_cordex_prep_rcp85_20712100.index.str.title()
hazard_cordex_prep_rcp85_20712100['average'] = pd.to_numeric(hazard_cordex_prep_rcp85_20712100['average'], errors='coerce')
hazard_cordex_prep_rcp85_20712100 = hazard_cordex_prep_rcp85_20712100.sort_index()
hazard_cordex_prep_rcp85_20712100 = hazard_cordex_prep_rcp85_20712100[['average']]
hazard_cordex_prep_rcp85_20712100

,average
Airports,
Bergamo Orio Al Serio,0.874555
Cagliari Elmas,0.092839
Catania Fontanarossa,0.052356
Milano Linate,0.691965
Milano Malpensa,0.753195
Napoli Capodichino,0.598805
Palermo Punta Raisi,0.316934
Roma Ciampino,0.833623
Roma Fiumicino,0.997940


In [61]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_prep_rcp85_20712100 = avg_df_exposure['average'] * hazard_cordex_prep_rcp85_20712100['average'] * vulnerability_df['average']
cordex_risk_prep_rcp85_20712100

# Convert the Series to a DataFrame
cordex_risk_prep_rcp85_20712100_df = pd.DataFrame(cordex_risk_prep_rcp85_20712100)

# Rename the column
cordex_risk_prep_rcp85_20712100_df.columns = ['Cordex Risk Prep rcp85 2071-2100']
cordex_risk_prep_rcp85_20712100_df

,Cordex Risk Prep rcp85 2071-2100
Bergamo Orio Al Serio,0.046477
Cagliari Elmas,0.007294
Catania Fontanarossa,0.001863
Milano Linate,0.066082
Milano Malpensa,0.360138
Napoli Capodichino,0.025628
Palermo Punta Raisi,0.018245
Roma Ciampino,0.058074
Roma Fiumicino,0.417627


In [62]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_prep_rcp85_20712100_df, "Cordex Risk Prep rcp85 2071-2100")

# Apply styling
styled_cordex_risk_prep_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_prep_df)

,Cordex Risk Prep rcp85 2071-2100,Quantile,Class
Bergamo Orio Al Serio,0.046477,0.555556,Medium
Cagliari Elmas,0.007294,0.222222,Very Low
Catania Fontanarossa,0.001863,0.111111,Very Low
Milano Linate,0.066082,0.777778,High
Milano Malpensa,0.360138,0.888889,Very High
Napoli Capodichino,0.025628,0.444444,Low
Palermo Punta Raisi,0.018245,0.333333,Low
Roma Ciampino,0.058074,0.666667,High
Roma Fiumicino,0.417627,1.000000,Very High
